UNDER DEVELOPMENT
- all seasons field-aware factorization machine

In [ ]:
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm

import plotly.express as px

In [ ]:
player_info = pd.read_csv('data/raw_pbp/playerlist.csv', index_col=[0])
season_files = sorted(glob('data/lineup_scores/*.pkl'))

In [ ]:
player_times = []
for season_file in tqdm(season_files):
    
    df = pd.read_pickle(season_file)

    player_time = pd.DataFrame()
    player_time['PERSON_ID'] = df[['off1', 'off2','off3', 'off4', 'off5', 'def1', 'def2', 'def3', 'def4', 'def5']].stack().values
    player_time['TIME'] = df.seconds.repeat(10).values
    player_time = player_time.groupby('PERSON_ID').TIME.sum().reset_index()
    player_time['season'] = season_file.split('/')[-1].split('.')[0]
    
    player_times.append(player_time)
    
player_times = pd.concat(player_times, ignore_index=True)

In [ ]:
all_pred_df = pd.read_pickle('data/ffm/all_pred_df.pkl')

In [ ]:
all_pred_df['weight'] = 1 / all_pred_df.cv_mse
pred_grp = all_pred_df.groupby(['PERSON_ID', 'season', 'poss']).apply(lambda x: (x['weight'] * x['pred']).sum() / x['weight'].sum()).reset_index()

In [ ]:
of = pred_grp[pred_grp.poss==0].copy()
df = pred_grp[pred_grp.poss==1].copy()

In [ ]:
of.rename(columns={0:'off'}, inplace=True)
df.rename(columns={0:'def'}, inplace=True)
of.drop(columns='poss', inplace=True)
df.drop(columns='poss', inplace=True)

In [ ]:
pred_grp = pd.merge(of, df, on=['PERSON_ID', 'season'])

In [ ]:
pred_grp = pred_grp.merge(player_times, on=['PERSON_ID', 'season'], how='left')

In [ ]:
pred_grp = pred_grp.merge(player_info[['DISPLAY_FIRST_LAST', 'PERSON_ID']], on='PERSON_ID', how='left')

In [ ]:
pred_career = pred_grp.groupby(['PERSON_ID', 'DISPLAY_FIRST_LAST']).agg({'off' : 'mean', 'def' : 'mean', 'TIME' : 'sum'}).reset_index()

In [ ]:
pred_career['contribution'] = np.exp(pred_career['off']) / np.exp(pred_career['def'])

In [ ]:
fig = px.scatter(pred_career, x="off", y="def", hover_name='DISPLAY_FIRST_LAST', size='TIME')

fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="red")
fig.add_hline(y=0, line_width=2, line_dash="dash", line_color="red")

fig.show()